# Laboratorium Nr 3
Zadaniem dla zespołów jest przeprowadzenie analizy danych i próba odpowiedzi na pytanie:„Czy na podstawie danych z ostatnich 24 lat można wyciągnąć wnioski na temat ocieplenia klimatu?”

In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
import pyodbc
from sqlalchemy import create_engine, text
from scipy.stats import linregress

## Przygotowanie danych

In [133]:
load_dotenv()

True

In [134]:

login = os.getenv("LOGIN")
password = os.getenv("PASSWORD")
db_host = os.getenv("DB_HOST")
port = os.getenv("PORT")
database = os.getenv("DB_NAME")

In [135]:
print(database)

Synop


In [136]:
conn_str = (
    f"mssql+pyodbc://{login}:{password}@{db_host},{port}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)


In [137]:
engine = create_engine(conn_str)

In [138]:
query = text("""
        SELECT TOP 200 *
        FROM Synop.dbo.Depesze as d
    """)



In [139]:

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

In [146]:
print(df.head(5))


   IDDepeszy  Stacja        Data   Godzina  \
0          1   12100  1999-03-18  09:00:00   
1          2   12105  1999-03-18  09:00:00   
2          3   12115  1999-03-18  09:00:00   
3          4   12120  1999-03-18  09:00:00   
4          5   12125  1999-03-18  09:00:00   

                                             Depesza  WysokoscPodstawyChmur  \
0  AAXX 18091 12100 42960 21803 10055 21011 30275...                   9999   
1  AAXX 18091 12105 41940 31502 10046 21005 30242...                   9999   
2  AAXX 18091 12115 41596 71903 10037 21005 30274...                    600   
3  AAXX 18091 12120 42997 71903 10041 21011 30284...                   9999   
4  AAXX 18091 12125 42997 61401 10054 21004 30240...                   9999   

   Widzialnosc  WielkoscZachmurzenia  KierunekWiatru  PredkoscWiatru  ...  \
0        10000                     2             180               3  ...   
1         4000                     3             150               2  ...   
2         4000   

In [144]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   IDDepeszy                     200 non-null    int64         
 1   Stacja                        200 non-null    int64         
 2   Data                          200 non-null    object        
 3   Godzina                       200 non-null    object        
 4   Depesza                       200 non-null    object        
 5   WysokoscPodstawyChmur         200 non-null    int64         
 6   Widzialnosc                   200 non-null    int64         
 7   WielkoscZachmurzenia          200 non-null    int64         
 8   KierunekWiatru                200 non-null    int64         
 9   PredkoscWiatru                200 non-null    int64         
 10  TemperaturaPowietrza          200 non-null    float64       
 11  TemperaturaPunktuRosy         20

In [143]:
print(df.describe())

        IDDepeszy        Stacja  WysokoscPodstawyChmur   Widzialnosc  \
count  200.000000    200.000000              200.00000    200.000000   
mean   100.500000  12377.635000             4097.13000  14660.500000   
min      1.000000  12100.000000                0.00000     25.000000   
25%     50.750000  12233.750000              600.00000  10000.000000   
50%    100.500000  12385.000000             1000.00000  10000.000000   
75%    150.250000  12500.000000             9999.00000  20000.000000   
max    200.000000  12695.000000             9999.00000  50000.000000   
std     57.879185    167.292846             4538.83036   9193.058256   

       WielkoscZachmurzenia  KierunekWiatru  PredkoscWiatru  \
count            200.000000      200.000000      200.000000   
mean               4.225000      143.400000        3.715000   
min                0.000000        0.000000        0.000000   
25%                1.000000      100.000000        2.000000   
50%                5.000000      140

In [141]:
missing_values = df.isnull().sum()
print("Missing values:\n", missing_values)

Missing values:
 IDDepeszy                       0
Stacja                          0
Data                            0
Godzina                         0
Depesza                         0
WysokoscPodstawyChmur           0
Widzialnosc                     0
WielkoscZachmurzenia            0
KierunekWiatru                  0
PredkoscWiatru                  0
TemperaturaPowietrza            0
TemperaturaPunktuRosy           0
CisnienieNaPoziomieStacji       0
CisnienieNaPoziomieMorza        0
ID_TendencjiCisnienia           0
WielkoscTendencjiCisnienia      0
WysokoscOpadu                   0
OkresPomiaruOpadu               0
ID_PogodyBiezacej               0
WielkoscZachmurzeniaNiskiego    0
ID_ChmurNiskich                 0
ID_ChmurSrednich                0
ID_ChmurWysokich                0
CzyMgla                         0
CzyMzawka                       0
CzyDeszcz                       0
CzySnieg                        0
CzyOpadMarznacy                 0
CzyPrzelotny                   

In [142]:
df = df[(df["temp"] > -40) & (df["temp"] < 50)]
df["rok"] = df["data_pomiaru"].dt.year
df["miesiac"] = df["data_pomiaru"].dt.month
df["dzien"] = df["data_pomiaru"].dt.day
df["godzina"] = df["data_pomiaru"].dt.hour

KeyError: 'temp'

In [ ]:
annual_mean = df.groupby(["rok", "nazwa_stacji"])["temp"].mean().reset_index()
annual_mean.rename(columns={"temp": "srednia_temp"}, inplace=True)
annual_mean_all = df.groupby("rok")["temp"].mean().reset_index()
annual_mean_all.rename(columns={"temp": "srednia_temp"}, inplace=True)

## Wizualizacja

In [ ]:
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

### Trend roczny (wszystkie stacje łącznie)

In [ ]:

plt.figure(figsize=(12, 6))
sns.lineplot(data=annual_mean_all, x="rok", y="srednia_temp", marker="o")
plt.title("Średnia roczna temperatura (2000-2023) - wszystkie stacje")
plt.xlabel("Rok")
plt.ylabel("Średnia temperatura (°C)")
plt.show()

### Trend roczny z podziałem na stacje


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=annual_mean, x="rok", y="srednia_temp", hue="nazwa_stacji", marker="o")
plt.title("Średnia roczna temperatura z podziałem na stacje (2000-2023)")
plt.xlabel("Rok")
plt.ylabel("Średnia temperatura (°C)")
plt.legend(title="Stacja")
plt.show()

### Boxplot roczny (rozkład temperatur w każdym roku)

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(data=df, x="rok", y="temp")
plt.title("Rozkład temperatur (2000-2023)")
plt.xlabel("Rok")
plt.ylabel("Temperatura (°C)")
plt.xticks(rotation=45)
plt.show()

### Analiza trendu

In [ ]:
annual_temp_series = annual_mean_all["srednia_temp"].values
years = annual_mean_all["rok"].values

In [ ]:
slope, intercept, r_value, p_value, std_err = linregress(years, annual_temp_series)
print(f"Slope (wzrost średniej temperatury na rok): {slope:.3f} °C/rok")
print(f"Intercept: {intercept:.3f} °C")
print(f"r-value: {r_value:.3f}, r^2: {r_value**2:.3f}")
print(f"p-value: {p_value:.5f}, std_err: {std_err:.3f}")